In [1]:
import pandas as pd
from sqlalchemy import create_engine
import os

In [2]:
from config.config_loader import Config

config = Config()

In [3]:
config_path = config.get_resolved_path("configs.config")
db_elec_path = config.get_resolved_path("data_path.elec.processed.facturas")
db_elec_manual_path = config.get_resolved_path("data_path.elec.processed.facturas_manual")
customers_path_single = config.get_resolved_path("data_path.elec.customers.folder")

In [4]:
# Parameters
config_path = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/configs/config.yaml"
db_elec_path = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/processed/elec/facturas_elec.db"
db_elec_manual_path = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/processed/elec/facturas_elec_manual.pkl"
customers_path_single = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/customers/elec"


In [5]:
from config.config_loader import Config

config = Config(config_file=config_path)

In [6]:
from sqlalchemy import create_engine

engine = create_engine(f'sqlite:///{db_elec_path}', echo=True)

facturas_df_sql = pd.read_sql(('SELECT * FROM facturas'), con=engine);

facturas_df_manual = pd.read_pickle(db_elec_manual_path)

# if not facturas_df[facturas_df['cups'] == cups].empty:

facturas_df = pd.concat([facturas_df_sql, facturas_df_manual], ignore_index=True)
for column in ['inicio_periodo', 'fin_periodo', 'issue_date']:
    facturas_df[column] = pd.to_datetime(facturas_df[column])

2025-08-25 11:05:29,458 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-25 11:05:29,459 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT * FROM facturas")
2025-08-25 11:05:29,459 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-08-25 11:05:29,459 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SELECT * FROM facturas")
2025-08-25 11:05:29,459 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-08-25 11:05:29,460 INFO sqlalchemy.engine.Engine SELECT * FROM facturas
2025-08-25 11:05:29,460 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-08-25 11:05:29,484 INFO sqlalchemy.engine.Engine ROLLBACK


In [7]:
import invoice_comp.facturas as facturas
Facturas = facturas.FacturasElec(df=facturas_df,language='en')

In [8]:
print(f"Customers found in database:\n{facturas_df['cliente'].unique()}")

Customers found in database:
['PR' 'Catalonia Fundacio' 'Kem One' 'Grupo Inspired' 'Sundolitt'
 'Trilla y Famesa' 'Altan Pharma' 'Taghleef' 'Montfrisa' 'VYOSA' 'PR2']


In [9]:
# potencia.columns.get_loc('potencia_p1') -> saca el numero de la columna
Facturas.get_measurement('potencia');
Facturas.get_measurement('energia');
Facturas.get_measurement('maximetros');
Facturas.get_invoice_title();
Facturas.get_invoices();

In [10]:
Facturas.group_facturas()
Facturas.set_period('facturas')
df_facturas = Facturas.facturas

In [11]:
clientes_elec = os.listdir(customers_path_single)
clientes_elec_clean = []
for cliente in clientes_elec:
    if not '.' in cliente:
        clientes_elec_clean.append(cliente)

In [12]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils.exceptions import InvalidFileException
from zipfile import BadZipFile

for cliente in clientes_elec_clean:

    excel_path = config.get_resolved_path('outputs.invoice_comp.elec.seguimiento', client_id=cliente)
    df_filtered = df_facturas[df_facturas['cliente'] == cliente]

    if not df_filtered.empty:
        print(f'Adding invoice data for {cliente}\n')

        try:
            # Just to verify the file is a valid Excel file before writing
            load_workbook(excel_path)
            
            with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                df_filtered.to_excel(writer, sheet_name='SQL', index=False)

        except (InvalidFileException, BadZipFile) as e:
            print(f"⚠️ Error: '{cliente}' - archivo dañado o inválido ({excel_path}). Se omite.\n")
            continue


Adding invoice data for Grupo Inspired

Adding invoice data for Taghleef

Adding invoice data for PR

Adding invoice data for Kem One

Adding invoice data for PR2

